In [1]:
# Clone the entire repo.
!git clone -l -s https://github.com/hse-aml/natural-language-processing.git cloned-repo
%cd cloned-repo
!ls

fatal: destination path 'cloned-repo' already exists and is not an empty directory.
/content/cloned-repo
AWS-tutorial.md  Docker-tutorial.md  README.md		    week2
common		 honor		     setup_google_colab.py  week3
docker		 project	     week1		    week4


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [7]:
!pip install tensorflow==1.14

     |████████████████████████████████| 109.2MB 96kB/s 
     |████████████████████████████████| 3.2MB 48.5MB/s 
     |████████████████████████████████| 491kB 41.3MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled tensorflow-estimator-2.2.0
  Found existing installation: tensorflow 2.2.0
    Uninstalling tensorflow-2.2.0:
      Successfully uninstalled tensorflow-2.2.0


In [3]:
tf.__version__

'1.14.0'

In [0]:
import random

In [0]:
def generate_equations(allowed_operators, dataset_size, min_value, max_value):
    """Generates pairs of equations and solutions to them.
    
       Each equation has a form of two integers with an operator in between.
       Each solution is an integer with the result of the operaion.
    
        allowed_operators: list of strings, allowed operators.
        dataset_size: an integer, number of equations to be generated.
        min_value: an integer, min value of each operand.
        max_value: an integer, max value of each operand.

        result: a list of tuples of strings (equation, solution).
    """
    sample = []
    for _ in range(dataset_size):
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        num1 = random.randint(min_value,max_value)
        num2 = random.randint(min_value,max_value)
        equa = num1+num2
        st = str(num1)+'+'+str(num2)
        sample.append((st,str(equa)))
        equa = num1-num2
        st = str(num1)+'-'+str(num2)
        sample.append((st,str(equa)))


    return sample

In [0]:
def test_generate_equations():
    allowed_operators = ['+', '-']
    dataset_size = 10
    for (input_, output_) in generate_equations(allowed_operators, dataset_size, 0, 100):
        if not (type(input_) is str and type(output_) is str):
            return "Both parts should be strings."
        if eval(input_) != int(output_):
            return "The (equation: {!r}, solution: {!r}) pair is incorrect.".format(input_, output_)
    return "Tests passed."

In [19]:
print(test_generate_equations())

Tests passed.


In [0]:
from sklearn.model_selection import train_test_split


In [0]:
allowed_operators = ['+', '-']
dataset_size = 100000
data = generate_equations(allowed_operators, dataset_size, min_value=0, max_value=9999)

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)

In [0]:
word2id = {symbol:i for i, symbol in enumerate('#^$+-1234567890')}
id2word = {i:symbol for symbol, i in word2id.items()}

In [0]:

start_symbol = '^'
end_symbol = '$'
padding_symbol = '#'

In [0]:
def sentence_to_ids(sentence, word2id, padded_len):
    """ Converts a sequence of symbols to a padded sequence of their ids.
    
      sentence: a string, input/output sequence of symbols.
      word2id: a dict, a mapping from original symbols to ids.
      padded_len: an integer, a desirable length of the sequence.

      result: a tuple of (a list of ids, an actual length of sentence).
    """
    
    sent_ids = []
    for i in range(len(sentence)):
      if i >= padded_len-1:
        break
      sent_ids.append(word2id[sentence[i]])
    
    sent_ids.append(word2id["$"])
    sent_len = len(sent_ids)
    for i in range(padded_len-len(sent_ids)):
      sent_ids.append(word2id["#"])
    
    
    return sent_ids, sent_len

In [0]:
def test_sentence_to_ids():
    sentences = [("123+123", 7), ("123+123", 8), ("123+123", 10)]
    expected_output = [([5, 6, 7, 3, 5, 6, 2], 7), 
                       ([5, 6, 7, 3, 5, 6, 7, 2], 8), 
                       ([5, 6, 7, 3, 5, 6, 7, 2, 0, 0], 8)] 
    for (sentence, padded_len), (sentence_ids, expected_length) in zip(sentences, expected_output):
        output, length = sentence_to_ids(sentence, word2id, padded_len)
        if output != sentence_ids:
            return("Convertion of '{}' for padded_len={} to {} is incorrect.".format(
                sentence, padded_len, output))
        if length != expected_length:
            return("Convertion of '{}' for padded_len={} has incorrect actual length {}.".format(
                sentence, padded_len, length))
    return("Tests passed.")

In [63]:

print(test_sentence_to_ids())

Tests passed.


In [0]:
def ids_to_sentence(ids, id2word):
    """ Converts a sequence of ids to a sequence of symbols.
    
          ids: a list, indices for the padded sequence.
          id2word:  a dict, a mapping from ids to original symbols.

          result: a list of symbols.
    """
 
    return [id2word[i] for i in ids]

In [0]:

def batch_to_ids(sentences, word2id, max_len):
    """Prepares batches of indices. 
    
       Sequences are padded to match the longest sequence in the batch,
       if it's longer than max_len, then max_len is used instead.

        sentences: a list of strings, original sequences.
        word2id: a dict, a mapping from original symbols to ids.
        max_len: an integer, max len of sequences allowed.

        result: a list of lists of ids, a list of actual lengths.
    """
    
    max_len_in_batch = min(max(len(s) for s in sentences) + 1, max_len)
    batch_ids, batch_ids_len = [], []
    for sentence in sentences:
        ids, ids_len = sentence_to_ids(sentence, word2id, max_len_in_batch)
        batch_ids.append(ids)
        batch_ids_len.append(ids_len)
    return batch_ids, batch_ids_len

In [0]:
def generate_batches(samples, batch_size=64):
    X, Y = [], []
    for i, (x, y) in enumerate(samples, 1):
        X.append(x)
        Y.append(y)
        if i % batch_size == 0:
            yield X, Y
            X, Y = [], []
    if X and Y:
        yield X, Y

In [67]:
sentences = train_set[0]
ids, sent_lens = batch_to_ids(sentences, word2id, max_len=10)
print('Input:', sentences)
print('Ids: {}\nSentences lengths: {}'.format(ids, sent_lens))

Input: ('855+9988', '10843')
Ids: [[12, 9, 9, 3, 13, 13, 12, 12, 2], [5, 14, 12, 8, 7, 2, 0, 0, 0]]
Sentences lengths: [9, 6]


In [0]:
class Seq2SeqModel(object):
    pass

In [0]:
def declare_placeholders(self):
    """Specifies placeholders for the model."""
    
    # Placeholders for input and its actual lengths.
    self.input_batch = tf.placeholder(shape=(None, None), dtype=tf.int32, name='input_batch')
    self.input_batch_lengths = tf.placeholder(shape=(None, ), dtype=tf.int32, name='input_batch_lengths')
    
    # Placeholders for groundtruth and its actual lengths.
    self.ground_truth = tf.placeholder(shape=(None,None),dtype=tf.int32,name="ground_truth")
    self.ground_truth_lengths = tf.placeholder(shape=(None,),dtype=tf.int32,name="ground_truth_lengths" )
        
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32,shape=[],name="learning_rate_ph")

In [0]:
Seq2SeqModel.__declare_placeholders = classmethod(declare_placeholders)

In [0]:
def create_embeddings(self, vocab_size, embeddings_size):
    """Specifies embeddings layer and embeds an input batch."""
     
    random_initializer = tf.random_uniform((vocab_size, embeddings_size), -1.0, 1.0)
    self.embeddings = tf.Variable(random_initializer,dtype=tf.float32,name="embeddings") 
    
    # Perform embeddings lookup for self.input_batch. 
    self.input_batch_embedded = tf.nn.embedding_lookup(self.embeddings,self.input_batch)

In [0]:
Seq2SeqModel.__create_embeddings = classmethod(create_embeddings)

In [0]:
def build_encoder(self, hidden_size):
    """Specifies encoder architecture and computes its output."""
    
    # Create GRUCell with dropout.
    encoder_cell = tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(num_units=hidden_size,dtype=tf.float32),input_keep_prob=self.dropout_ph)
    
    # Create RNN with the predefined cell.
    _, self.final_encoder_state = tf.nn.dynamic_rnn(encoder_cell,self.input_batch_embedded,sequence_length=self.input_batch_lengths,dtype=tf.float32)

In [0]:
Seq2SeqModel.__build_encoder = classmethod(build_encoder)

In [0]:
def build_decoder(self, hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id):
    """Specifies decoder architecture and computes the output.
    
        Uses different helpers:
          - for train: feeding ground truth
          - for inference: feeding generated output

        As a result, self.train_outputs and self.infer_outputs are created. 
        Each of them contains two fields:
          rnn_output (predicted logits)
          sample_id (predictions).

    """
    
    # Use start symbols as the decoder inputs at the first time step.
    batch_size = tf.shape(self.input_batch)[0]
    start_tokens = tf.fill([batch_size], start_symbol_id)
    ground_truth_as_input = tf.concat([tf.expand_dims(start_tokens, 1), self.ground_truth], 1)
    
    # Use the embedding layer defined before to lookup embedings for ground_truth_as_input. 
    self.ground_truth_embedded =tf.nn.embedding_lookup(self.embeddings,ground_truth_as_input)
     
    # Create TrainingHelper for the train stage.
    train_helper = tf.contrib.seq2seq.TrainingHelper(self.ground_truth_embedded, 
                                                     self.ground_truth_lengths)
    
    # Create GreedyEmbeddingHelper for the inference stage.
    # You should provide the embedding layer, start_tokens and index of the end symbol.
    infer_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(self.embeddings,start_tokens=start_tokens,end_token=end_symbol_id)
    
  
    def decode(helper, scope, reuse=None):
        """Creates decoder and return the results of the decoding with a given helper."""
        
        with tf.variable_scope(scope, reuse=reuse):
            # Create GRUCell with dropout. Do not forget to set the reuse flag properly.
            decoder_cell = tf.contrib.rnn.DropoutWrapper(tf.nn.rnn_cell.GRUCell(hidden_size,reuse=reuse),dtype=tf.float32,input_keep_prob=self.dropout_ph)
            
            # Create a projection wrapper.
            decoder_cell = tf.contrib.rnn.OutputProjectionWrapper(decoder_cell, vocab_size, reuse=reuse)
            
            # Create BasicDecoder, pass the defined cell, a helper, and initial state.
            # The initial state should be equal to the final state of the encoder!
            decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,helper=helper,initial_state=self.final_encoder_state)
            
            # The first returning argument of dynamic_decode contains two fields:
            #   rnn_output (predicted logits)
            #   sample_id (predictions)
            outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder=decoder, maximum_iterations=max_iter, 
                                                              output_time_major=False, impute_finished=True)

            return outputs
        
    self.train_outputs = decode(train_helper, 'decode')
    self.infer_outputs = decode(infer_helper, 'decode', reuse=True)


In [0]:
Seq2SeqModel.__build_decoder = classmethod(build_decoder)

In [0]:
def compute_loss(self):
    """Computes sequence loss (masked cross-entopy loss with logits)."""
    
    weights = tf.cast(tf.sequence_mask(self.ground_truth_lengths), dtype=tf.float32)
    
    self.loss = tf.contrib.seq2seq.sequence_loss(self.train_outputs.rnn_output,self.ground_truth,weights)

In [0]:
Seq2SeqModel.__compute_loss = classmethod(compute_loss)

In [0]:

def perform_optimization(self):
    """Specifies train_op that optimizes self.loss."""
    
    self.train_op = tf.contrib.layers.optimize_loss(self.loss,global_step=tf.train.get_global_step(),
                                                    learning_rate=self.learning_rate_ph,clip_gradients=1.0,optimizer="Adam")



In [0]:
Seq2SeqModel.__perform_optimization = classmethod(perform_optimization)

In [0]:
def init_model(self, vocab_size, embeddings_size, hidden_size, 
               max_iter, start_symbol_id, end_symbol_id, padding_symbol_id):
    
    self.__declare_placeholders()
    self.__create_embeddings(vocab_size, embeddings_size)
    self.__build_encoder(hidden_size)
    self.__build_decoder(hidden_size, vocab_size, max_iter, start_symbol_id, end_symbol_id)
    
    # Compute loss and back-propagate.
    self.__compute_loss()
    self.__perform_optimization()
    
    # Get predictions for evaluation.
    self.train_predictions = self.train_outputs.sample_id
    self.infer_predictions = self.infer_outputs.sample_id

In [0]:
Seq2SeqModel.__init__ = classmethod(init_model)

In [0]:
def train_on_batch(self, session, X, X_seq_len, Y, Y_seq_len, learning_rate, dropout_keep_probability):
    feed_dict = {
            self.input_batch: X,
            self.input_batch_lengths: X_seq_len,
            self.ground_truth: Y,
            self.ground_truth_lengths: Y_seq_len,
            self.learning_rate_ph: learning_rate,
            self.dropout_ph: dropout_keep_probability
        }
    pred, loss, _ = session.run([
            self.train_predictions,
            self.loss,
            self.train_op], feed_dict=feed_dict)
    return pred, loss

In [0]:
Seq2SeqModel.train_on_batch = classmethod(train_on_batch)

In [0]:
def predict_for_batch(self, session, X, X_seq_len):
    feed_dict = {
        self.input_batch:X,
        self.input_batch_lengths:X_seq_len
    }
    pred = session.run([
            self.infer_predictions
        ], feed_dict=feed_dict)[0]
    return pred

def predict_for_batch_with_loss(self, session, X, X_seq_len, Y, Y_seq_len):
    feed_dict = {
        self.input_batch:X,
        self.input_batch_lengths:X_seq_len,
        self.ground_truth: Y,
        self.ground_truth_lengths:Y_seq_len
    }
    pred, loss = session.run([
            self.infer_predictions,
            self.loss,
        ], feed_dict=feed_dict)
    return pred, loss

In [0]:
Seq2SeqModel.predict_for_batch = classmethod(predict_for_batch)
Seq2SeqModel.predict_for_batch_with_loss = classmethod(predict_for_batch_with_loss)

In [115]:
tf.reset_default_graph()

model = Seq2SeqModel(vocab_size=len(word2id),embeddings_size=20,hidden_size=512,max_iter=7,start_symbol_id=word2id[start_symbol],
                     end_symbol_id=word2id[end_symbol],padding_symbol_id=word2id[padding_symbol])

batch_size = 128
n_epochs = 10
learning_rate = 0.001
dropout_keep_probability = 0.5
max_len = 20

n_step = int(len(train_set) / batch_size)

In [116]:
session = tf.Session()
session.run(tf.global_variables_initializer())
            
invalid_number_prediction_counts = []
all_model_predictions = []
all_ground_truth = []

print('Start training... \n')
for epoch in range(n_epochs):  
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    print('Train: epoch', epoch + 1)
    for n_iter, (X_batch, Y_batch) in enumerate(generate_batches(train_set, batch_size=batch_size)):
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        # prepare the data (X_batch and Y_batch) for training
        # using function batch_to_ids

        X, X_seq_len = batch_to_ids(X_batch, word2id, max_len)
        Y, Y_seq_len = batch_to_ids(Y_batch, word2id, max_len)
        predictions, loss = model.train_on_batch(session, 
                                                X, 
                                                X_seq_len, 
                                                Y, 
                                                Y_seq_len, 
                                                learning_rate, 
                                                dropout_keep_probability)
        
        if n_iter % 200 == 0:
            print("Epoch: [%d/%d], step: [%d/%d], loss: %f" % (epoch + 1, n_epochs, n_iter + 1, n_step, loss))
                
    X_sent, Y_sent = next(generate_batches(test_set, batch_size=batch_size))
    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    # prepare test data (X_sent and Y_sent) for predicting 
    # quality and computing value of the loss function
    # using function batch_to_ids
    X, X_seq_len = batch_to_ids(X_sent, word2id, max_len)
    Y, Y_seq_len = batch_to_ids(Y_sent, word2id, max_len)
    predictions, loss = model.predict_for_batch_with_loss(session, 
                                                X, 
                                                X_seq_len, 
                                                Y, 
                                                Y_seq_len) 
    
    print('Test: epoch', epoch + 1, 'loss:', loss,)
    for x, y, p  in list(zip(X, Y, predictions))[:3]:
        print('X:',''.join(ids_to_sentence(x, id2word)))
        print('Y:',''.join(ids_to_sentence(y, id2word)))
        print('O:',''.join(ids_to_sentence(p, id2word)))
        print('')

    model_predictions = []
    ground_truth = []
    invalid_number_prediction_count = 0
    # For the whole test set calculate ground-truth values (as integer numbers)
    # and prediction values (also as integers) to calculate metrics.
    # If generated by model number is not correct (e.g. '1-1'), 
    # increase invalid_number_prediction_count and don't append this and corresponding
    # ground-truth value to the arrays.
    for X_batch, Y_batch in generate_batches(test_set, batch_size=batch_size):
        X, X_seq_len = batch_to_ids(X_batch, word2id, max_len)
        Y, Y_seq_len = batch_to_ids(Y_batch, word2id, max_len)
        predictions, loss = model.predict_for_batch_with_loss(session, X, X_seq_len, Y, Y_seq_len)
    
    all_model_predictions.append(model_predictions)
    all_ground_truth.append(ground_truth)
    invalid_number_prediction_counts.append(invalid_number_prediction_count)
            
print('\n...training finished.')

Start training... 

Train: epoch 1
Epoch: [1/10], step: [1/1250], loss: 2.705241
Epoch: [1/10], step: [201/1250], loss: 1.802342
Epoch: [1/10], step: [401/1250], loss: 1.663049
Epoch: [1/10], step: [601/1250], loss: 1.631186
Epoch: [1/10], step: [801/1250], loss: 1.553585
Epoch: [1/10], step: [1001/1250], loss: 1.521577
Epoch: [1/10], step: [1201/1250], loss: 1.452955
Test: epoch 1 loss: 1.3925002
X: 2626+4270$
Y: 6896$#
O: 6521$#

X: 6486+5330$
Y: 11816$
O: 12224$

X: 9699-1318$
Y: 8381$#
O: 8231$#

Train: epoch 2
Epoch: [2/10], step: [1/1250], loss: 1.496710
Epoch: [2/10], step: [201/1250], loss: 1.431549
Epoch: [2/10], step: [401/1250], loss: 1.430471
Epoch: [2/10], step: [601/1250], loss: 1.419047
Epoch: [2/10], step: [801/1250], loss: 1.394574
Epoch: [2/10], step: [1001/1250], loss: 1.364852
Epoch: [2/10], step: [1201/1250], loss: 1.353233
Test: epoch 2 loss: 1.3016651
X: 6373+5079$
Y: 11452$
O: 11271$

X: 5261+7583$
Y: 12844$
O: 12270$

X: 4630+9111$
Y: 13741$
O: 13400$

Train: e